# Fine tuning


## Import


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import tensorflow_datasets as tfds
from keras.preprocessing import image

from tensorflow.keras.layers import Conv2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau



import warnings
warnings.filterwarnings("ignore")

In [2]:
%cd /content/drive/MyDrive/ai_prj/prj_ImageSuper-Resolution/


/content/drive/MyDrive/ai_prj/prj_ImageSuper-Resolution


In [3]:
## Load dataset

In [4]:
ds_train = tfds.load('div2k/unknown_x4',split='train[:75%]',as_supervised=True)
ds_valid = tfds.load('div2k/unknown_x4',split='train[75%:]',as_supervised=True)
ds_test = tfds.load('div2k/unknown_x4',split='validation',as_supervised=True)


import agumentation
train = agumentation.preprocessing(ds_train,scale=4)
valid = agumentation.preprocessing(ds_valid,scale=4,batch_size=1,repeat_count=1,random_transform=False)

test  = agumentation.preprocessing(ds_test,scale=4,batch_size=1,repeat_count=1,random_transform=False)


EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_unknown_X4.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_unknown_X4.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

...

  0%|          | 0/800 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/div2k/unknown_x4/2.0.0.incomplete0HFKEF/div2k-validation.tfrecord


  0%|          | 0/100 [00:00<?, ? examples/s]

Dataset div2k downloaded and prepared to /root/tensorflow_datasets/div2k/unknown_x4/2.0.0. Subsequent calls will reuse this data.


In [5]:
## Load model
def PSNR(super_resolution, high_resolution):
    """Compute the peak signal-to-noise ratio, measures quality of image."""
    # Max value of pixel is 255
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val=255)[0]
    return psnr_value

base = tf.keras.models.load_model("save_model/EDSRx2.h5",custom_objects={"PSNR": PSNR})

In [6]:
## Load weight
base.load_weights("checkpoint/edsr/cp1.h5")

In [7]:
## Headmodel
from model import edsr 
x = base.layers[-2].output
head = edsr.upsample(x,2,num_filters=64)
head = Conv2D(3, 3, padding='same')(head)

head = Lambda(edsr.denormalize)(head)


In [8]:
for layer in base.layers:
    layer._name = str("base")+ layer.name 

In [9]:
## Full model

model = Model(inputs=base.input, outputs=head)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 baseinput_1 (InputLayer)       [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 baselambda (Lambda)            (None, None, None,   0           ['baseinput_1[0][0]']            
                                3)                                                                
                                                                                                  
 baseconv2d (Conv2D)            (None, None, None,   1792        ['baselambda[0][0]']             
                                64)                                                           

In [10]:
for layer in base.layers:
  layer.trainable = False

In [14]:

optim = Adam(lr=0.1)
model.compile(optimizer= optim , loss = 'mse', metrics=[PSNR])
# early = EarlyStopping(monitor = 'val_loss', patience = 5)
reducelr = ReduceLROnPlateau( monitor='val_loss',factor=tf.math.exp(-0.1), patience=3, verbose=1, mode='auto', min_lr=1e-5)
checkpoint = ModelCheckpoint("checkpoint/edsr/cp_pre.h5", monitor="val_loss", verbose=0, save_best_only=True,save_weight_only = True, mode="auto",)

## Training

In [16]:
model.fit(train,validation_data=valid,epochs=100,verbose=True, callbacks=[checkpoint,reducelr],steps_per_epoch=200)

Epoch 1/100
200/200 [==============================] - 233s 1s/step - loss: 675.7160 - PSNR: 22.2492 - val_loss: 488.9443 - val_PSNR: 22.2678 - lr: 0.1000
Epoch 2/100
200/200 [==============================] - 232s 1s/step - loss: 604.0790 - PSNR: 24.1124 - val_loss: 475.1469 - val_PSNR: 22.4073 - lr: 0.1000
Epoch 3/100
200/200 [==============================] - 230s 1s/step - loss: 567.6514 - PSNR: 23.2937 - val_loss: 472.4155 - val_PSNR: 22.4379 - lr: 0.1000
Epoch 4/100
200/200 [==============================] - 235s 1s/step - loss: 566.7604 - PSNR: 23.9794 - val_loss: 471.6462 - val_PSNR: 22.4076 - lr: 0.1000
Epoch 5/100
200/200 [==============================] - 226s 1s/step - loss: 582.3988 - PSNR: 24.1087 - val_loss: 471.6814 - val_PSNR: 22.4360 - lr: 0.1000
Epoch 6/100
200/200 [==============================] - 230s 1s/step - loss: 586.8701 - PSNR: 23.3181 - val_loss: 661.1481 - val_PSNR: 20.4408 - lr: 0.1000
Epoch 7/100
200/200 [==============================] - ETA: 0s - loss:

KeyboardInterrupt: ignored